# Applying a CNN (built with TensorFlow) to an image dataset

Welcome to part 2 of PA3! In this part, you'll implement:
- helper functions that you will use when implementing a TensorFlow model
- a fully functional CNN using TensorFlow 

**After this assignment you will be able to:**
- Build and train a CNNs in TensorFlow for classification tasks 

Some of this assignment will require becoming familiar with TensorFlow – there will also be a tutorial given (in class) on TensorFlow/Keras during the time this assignment is posted.

you will need to install tensorflow in your system first. If your system does not support GPU, you may want to install cpu-only version of tensorflow for this assignment on your laptop.

Check:
https://www.tensorflow.org/install/ 

---
## 1.0 - TensorFlow model

In the previous assignment, you built helper functions using `numpy` to understand how basic operations work in CNNs. However, most practical applications of deep learning today are built using programming frameworks, which have many built-in functions you can simply call. Therefore, you do not need to implement those operations if you are using one of those popular frameworks (such as tensorflow).

As usual, we will start by loading in the packages. 

In [ ]:
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import scipy
from PIL import Image
from scipy import ndimage
import tensorflow as tf
from tensorflow.python.framework import ops
from utils import *

%matplotlib inline
np.random.seed(1)

Run the next cell and load the "SIGNS" dataset.

In [ ]:
# Loading the data (signs)
X_train_orig, Y_train_orig, X_test_orig, Y_test_orig, classes = dataloader()

As a reminder, the SIGNS dataset is a collection of 6 signs (from American Sign Language, ASL) representing numbers from 0 to 5.

<img src="images/SIGNS.png" style="width:800px;height:300px;">

The next cell will show you an example of a labelled image in the dataset. Feel free to change the value of `index` below and re-run to see different examples. 

In [ ]:
# Example of a picture
index = 42
plt.imshow(X_train_orig[index])
print ("y = " + str(np.squeeze(Y_train_orig[:, index])))

In PA2, we built a logistic regression model (think of it like a simplified neural network) to classify cat/non-cat. The general ideas and procedures from that assignment will make appearances quite often, here. However, as these datasets are images, CNNs are much more effective than NNs and logistic regression models.

To get started, let's examine the shapes of your data. 

In [ ]:
X_train = X_train_orig/255.
X_test = X_test_orig/255.
Y_train = convert_to_one_hot(Y_train_orig, 6).T
Y_test = convert_to_one_hot(Y_test_orig, 6).T
print ("number of training examples = " + str(X_train.shape[0]))
print ("number of test examples = " + str(X_test.shape[0]))
print ("X_train shape: " + str(X_train.shape))
print ("Y_train shape: " + str(Y_train.shape))
print ("X_test shape: " + str(X_test.shape))
print ("Y_test shape: " + str(Y_test.shape))
conv_layers = {}

### 1.1 - Create placeholders

TensorFlow requires the use of `tf.placeholder`s for data which will be supplied to the model when it's being run – think of these `tf.placeholder`s as promises to the model you'll provide what it expects.

<div class="alert alert-info"><h1>Exercise 1</h1>
    <p>Implement <code>create_placholders</code> to provide placeholders for the input image <code>X</code> and the output <code>Y</code>. You shouldn't define the number of training examples for the moment. To do so, you could use <code>None</code> as the batch size – this gives you the flexibility to choose it later. Hence <code>X</code> should be of dimension <code>[None, n_H0, n_W0, n_C0]</code> and <code>Y</code> should be of dimension <code>[None, n_y]</code>. <a href="https://www.tensorflow.org/api_docs/python/tf/placeholder">Hint</a>.
</div>

In [ ]:
# GRADED FUNCTION: create_placeholders

def create_placeholders(n_H0, n_W0, n_C0, n_y):
    """
    Creates the placeholders for the tensorflow session.
    
    Arguments:
    n_H0 -- scalar, height of an input image
    n_W0 -- scalar, width of an input image
    n_C0 -- scalar, number of channels of the input
    n_y -- scalar, number of classes
        
    Returns:
    X -- placeholder for the data input, of shape [None, n_H0, n_W0, n_C0] and dtype "float"
    Y -- placeholder for the input labels, of shape [None, n_y] and dtype "float"
    """

    ### START CODE HERE ### (~2 lines)
    X = None
    Y = None
    ### END CODE HERE ###
    
    return X, Y

In [ ]:
X, Y = create_placeholders(64, 64, 3, 6)
print ("X = " + str(X))
print ("Y = " + str(Y))

**Expected Output**

```python
X = Tensor("Placeholder:0", shape=(?, 64, 64, 3), dtype=float32)
Y = Tensor("Placeholder_1:0", shape=(?, 6), dtype=float32)
```

### 1.2 - Initialize parameters

You'll initialize weights/filters $W1$ and $W2$ using `tf.contrib.layers.xavier_initializer(seed=0)`. Don't worry about bias as you will soon see that TensorFlow functions take care of the bias. Also note that you'll only initialize the weights/filters for the `conv2d` functions. TensorFlow initializes layers for the fully connected networks (vanilla NNs) automatically. We'll talk more about that later in this assignment.

<div class="alert alert-info"><h1>Exercise 2</h1>
<p>Implement `initialize_parameters()`. The dimensions for each group of filters are provided below.</p>
</div>

To initialize a parameter $W$ with shape [1,2,3,4] in Tensorflow, use:
```python
W = tf.get_variable("W", [1,2,3,4], initializer=...)
```
[More Info](https://www.tensorflow.org/api_docs/python/tf/get_variable).

In [ ]:
# GRADED FUNCTION: initialize_parameters

def initialize_parameters():
    """
    Initializes weight parameters to build a neural network with tensorflow. The shapes are:
                        W1 : [4, 4, 3, 8]
                        W2 : [2, 2, 8, 16]
    Returns:
    parameters -- a dictionary of tensors containing W1, W2
    """
    
    tf.set_random_seed(1)                              # so that your "random" numbers match ours
        
    ### START CODE HERE ### (~2 lines of code)
    W1 = None
    W2 = None
    ### END CODE HERE ###

    parameters = {"W1": W1,
                  "W2": W2}
    
    return parameters

In [ ]:
tf.reset_default_graph()
with tf.Session() as sess_test:
    parameters = initialize_parameters()
    init = tf.global_variables_initializer()
    sess_test.run(init)
    print("W1 = " + str(parameters["W1"].eval()[1,1,1]))
    print("W2 = " + str(parameters["W2"].eval()[1,1,1]))

**Expected Output:**
```python
W1 = [ 0.00131723  0.14176141 -0.04434952  0.09197326  0.14984085 -0.03514394
 -0.06847463  0.05245192]
W2 = [-0.08566415  0.17750949  0.11974221  0.16773748 -0.0830943  -0.08058
 -0.00577033 -0.14643836  0.24162132 -0.05857408 -0.19055021  0.1345228
 -0.22779644 -0.1601823  -0.16117483 -0.10286498]
```

### 1.2 - Forward propagation

In TensorFlow, there are built-in functions which carry out the convolution steps for you:
- `tf.nn.conv2d(X, W1, strides=[1, s, s, 1], padding='SAME')`: given an input $X$ and a group of filters $W1$,  convolve $W1$'s filters on $X$. The third input ([1,f,f,1]) represents the strides for each dimension of the input `(m, n_H_prev, n_W_prev, n_C_prev)`. You can read the full documentation [here](https://www.tensorflow.org/api_docs/python/tf/nn/conv2d)
- `tf.nn.max_pool(A, ksize=[1, f, f, 1], strides=[1, s, s, 1], padding='SAME')`: given an input $A$, this function uses a window of size $(f, f)$ and strides of size $(s, s)$ to carry out max pooling over each window. You can read the full documentation [here](https://www.tensorflow.org/api_docs/python/tf/nn/max_pool)
- `tf.nn.relu(Z1)`: compute the elementwise `ReLU` of $Z1$ (which can be any shape). You can read the full documentation [here.](https://www.tensorflow.org/api_docs/python/tf/nn/relu)
- `tf.contrib.layers.flatten(P)`: given an input $P$, flatten each example into a 1D vector it while maintaining the batch-size. It returns a flattened tensor with shape `[batch_size, k]`. You can read the full documentation [here.](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/flatten)
- `tf.contrib.layers.fully_connected(F, num_outputs):` given a flattened input $F$, compute the output using a fully connected layer. You can read the full documentation [here.](https://www.tensorflow.org/api_docs/python/tf/contrib/layers/fully_connected)

In `tf.contrib.layers.fully_connected`, the FCL automatically initializes weights in the graph and keeps training them as yoru train the model. Becasue of this, it's unnecessary to initial the weights, here, when initializing the parameters.

<div class="alert alert-info"><h1>Exercise 3</h1>
    <p>Implement <code>forward_propagation</code> to build the following model: <code>conv2d -> relu -> max-pool -> conv2d -> relu -> max-pool -> flatten -> fully-connected</code>. You should use the functions above. </p>
</div>

In detail, we will use the following parameters for all the steps:
  - `conv2d`: stride 1, padding is "SAME"
  - `relu`
  - `max-pool`: Use an 8 by 8 filter size and an 8 by 8 stride, padding is "SAME"
  - `conv2d`: stride 1, padding is "SAME"
  - `relu`
  - `max-pool`: Use a 4 by 4 filter size and a 4 by 4 stride, padding is "SAME"
  - `flatten` the previous output.
  - `fully-connected` (FC) layer: Apply a fully connected layer without an non-linear activation function. Do **not** call the softmax here. There should be 6 uunits in the output layer, each of which will pass their outputs to a $softmax$ later on. TensorFlow lumps the softmax and cost fnctions together, which we'll call in a later function where we compute the cost.

In [ ]:
# GRADED FUNCTION: forward_propagation

def forward_propagation(X, parameters):
    """
    Implements the forward propagation for the model:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    Arguments:
    X -- input dataset placeholder, of shape (input size, number of examples)
    parameters -- python dictionary containing your parameters "W1", "W2"
                  the shapes are given in initialize_parameters

    Returns:
    Z3 -- the output of the last LINEAR unit
    """
    
    # Retrieve the parameters from the dictionary "parameters" 
    W1 = parameters['W1']
    W2 = parameters['W2']
    
    ### START CODE HERE ###
    # conv2d: stride of 1, padding 'SAME'
    Z1 = None
    # relu
    A1 = None
    # max-pool: window 8x8, stride 8, padding 'SAME'
    P1 = None
    # conv2d: filters W2, stride 1, padding 'SAME'
    Z2 = None
    # relu
    A2 = None
    # max-pool: window 4x4, stride 4, padding 'SAME'
    P2 = None
    # flatten
    P = None
    # fully-connected without non-linear activation function (not not call softmax).
    # 6 neurons in output layer. Hint: one of the arguments should be "activation_fn=None" 
    Z3 = None
    ### END CODE HERE ###

    return Z3

In [ ]:
tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(1)
    X, Y = create_placeholders(64, 64, 3, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    init = tf.global_variables_initializer()
    sess.run(init)
    a = sess.run(Z3, {X: np.random.randn(2,64,64,3), Y: np.random.randn(2,6)})
    print("Z3 = " + str(a))

**Expected Output**:

```python
Z3 = [[-0.44670227 -1.57208765 -1.53049231 -2.31013036 -1.29104376  0.46852064]
 [-0.17601591 -1.57972014 -1.4737016  -2.61672091 -1.00810647  0.5747785 ]]
```

### 1.3 - Compute cost

Implement the compute cost function below. You might find these two functions helpful: 

- `tf.nn.softmax_cross_entropy_with_logits(logits=Z3, labels=Y)`: computes the "softmax entropy loss." This function both computes the $softmax$ activation function as well as the resulting loss. You can check the full documentation [here.](https://www.tensorflow.org/api_docs/python/tf/nn/softmax_cross_entropy_with_logits)
- `tf.reduce_mean`: computes the mean of elements across dimensions of a tensor. Use this to sum the losses over all the examples to get the overall cost. You can check the full documentation [here.](https://www.tensorflow.org/api_docs/python/tf/reduce_mean)

<div class="alert alert-info"><h1>Exercise 4</h1>
    <p>Compute the cost below using the function above.</p>
</div>

In [ ]:
# GRADED FUNCTION: compute_cost 

def compute_cost(Z3, Y):
    """
    Computes the cost
    
    Arguments:
    Z3 -- output of forward propagation (output of the last LINEAR unit), of shape (6, number of examples)
    Y -- "true" labels vector placeholder, same shape as Z3
    
    Returns:
    cost - Tensor of the cost function
    """
    
    ### START CODE HERE ### (~1 line of code)
    cost = None
    ### END CODE HERE ###
    
    return cost

In [ ]:
tf.reset_default_graph()

with tf.Session() as sess:
    np.random.seed(1)
    X, Y = create_placeholders(64, 64, 3, 6)
    parameters = initialize_parameters()
    Z3 = forward_propagation(X, parameters)
    cost = compute_cost(Z3, Y)
    init = tf.global_variables_initializer()
    sess.run(init)
    a = sess.run(cost, {X: np.random.randn(4,64,64,3), Y: np.random.randn(4,6)})
    print("cost = " + str(a))

**Expected Output**: 
```python
cost = 2.91034
```

## 1.4 Model 

Finally, we can merge the helper fnctions from earlier into a model!  You'll then train this on the SIGNS dataset.

You have implemented `random_mini_batches()` in the Optimization programming assignment of course 2. Remember that this function returns a list of mini-batches.

<div class="alert alert-info"><h1>Exercise 5</h1>
    <p>Implement <code>model</code> below. It should:</p>
    <ul>
        <li> create placholders </li>
        <li> initialize parameters </li>
        <li> forward propagate </li>
        <li> compute the cost </li>
        <li> create an optimizer </li>
    </ul>
</div>

Finally, you'' create a `tf.Session()` and run a `for`-loop  for `num_epochs`, get the mini-batches, and then for each mini-batch you will optimize the function. [Hint for initializing the variables](https://www.tensorflow.org/api_docs/python/tf/global_variables_initializer)

In [ ]:
# GRADED FUNCTION: model

def model(X_train, Y_train, X_test, Y_test, learning_rate=0.009,
          num_epochs=100, minibatch_size=64, print_cost=True):
    """
    Implements a three-layer ConvNet in Tensorflow:
    CONV2D -> RELU -> MAXPOOL -> CONV2D -> RELU -> MAXPOOL -> FLATTEN -> FULLYCONNECTED
    
    Arguments:
    X_train -- training set, of shape (None, 64, 64, 3)
    Y_train -- test set, of shape (None, n_y = 6)
    X_test -- training set, of shape (None, 64, 64, 3)
    Y_test -- test set, of shape (None, n_y = 6)
    learning_rate -- learning rate of the optimization
    num_epochs -- number of epochs of the optimization loop
    minibatch_size -- size of a minibatch
    print_cost -- True to print the cost every 100 epochs
    
    Returns:
    train_accuracy -- real number, accuracy on the train set (X_train)
    test_accuracy -- real number, testing accuracy on the test set (X_test)
    parameters -- parameters learnt by the model. They can then be used to predict.
    """
    
    ops.reset_default_graph()                         # to be able to rerun the model without overwriting tf variables
    tf.set_random_seed(1)                             # to keep results consistent (tensorflow seed)
    seed = 3                                          # to keep results consistent (numpy seed)
    (m, n_H0, n_W0, n_C0) = X_train.shape             
    n_y = Y_train.shape[1]                            
    costs = []                                        # To keep track of the cost
    
    # Create Placeholders of the correct shape
    ### START CODE HERE ### (1 line)
    X, Y = create_placeholders(n_H0, n_W0, n_C0, n_y)
    ### END CODE HERE ###

    # Initialize parameters
    ### START CODE HERE ### (1 line)
    parameters = initialize_parameters()
    ### END CODE HERE ###
    
    # Forward propagation: Build the forward propagation in the tensorflow graph
    ### START CODE HERE ### (1 line)
    Z3 = forward_propagation(X, parameters)
    ### END CODE HERE ###
    
    # Cost function: Add cost function to tensorflow graph
    ### START CODE HERE ### (1 line)
    cost = compute_cost(Z3, Y)
    ### END CODE HERE ###
    
    # Backpropagation: Define the tensorflow optimizer. Use an AdamOptimizer that minimizes the cost.
    ### START CODE HERE ### (1 line)
    optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
    ### END CODE HERE ###
    
    # Initialize all the variables globally
    init = tf.global_variables_initializer()
     
    # Start the session to compute the tensorflow graph
    with tf.Session() as sess:
        
        # Run the initialization
        sess.run(init)
        
        # Do the training loop
        for epoch in range(num_epochs):

            minibatch_cost = 0.
            num_minibatches = int(m / minibatch_size) # number of minibatches of size minibatch_size in the train set
            seed = seed + 1
            minibatches = random_mini_batches(X_train, Y_train, minibatch_size, seed)

            for minibatch in minibatches:

                # Select a minibatch
                (minibatch_X, minibatch_Y) = minibatch
                # IMPORTANT: The line that runs the graph on a minibatch.
                # Run the session to execute the optimizer and the cost, the feedict should contain a minibatch for (X,Y).
                ### START CODE HERE ### (1 line)
                _ , temp_cost = sess.run([optimizer, cost], feed_dict={X:minibatch_X, Y:minibatch_Y})
                ### END CODE HERE ###
                
                minibatch_cost += temp_cost / num_minibatches
                

            # Print the cost every epoch
            if print_cost == True and epoch % 5 == 0:
                print ("Cost after epoch %i: %f" % (epoch, minibatch_cost))
            if print_cost == True and epoch % 1 == 0:
                costs.append(minibatch_cost)
        
        
        # plot the cost
        plt.plot(np.squeeze(costs))
        plt.ylabel('cost')
        plt.xlabel('iterations (per tens)')
        plt.title("Learning rate =" + str(learning_rate))
        plt.show()

        # Calculate the correct predictions
        predict_op = tf.argmax(Z3, 1)
        correct_prediction = tf.equal(predict_op, tf.argmax(Y, 1))
        
        # Calculate accuracy on the test set
        accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
        print(accuracy)
        train_accuracy = accuracy.eval({X: X_train, Y: Y_train})
        test_accuracy = accuracy.eval({X: X_test, Y: Y_test})
        print("Train Accuracy:", train_accuracy)
        print("Test Accuracy:", test_accuracy)
                
        return train_accuracy, test_accuracy, parameters

Run the following cell to train your model for 100 epochs. Check if your cost after epoch 0 and 5 matches our output. If not, stop the cell and go back to your code!

In [ ]:
_, _, parameters = model(X_train, Y_train, X_test, Y_test)

**Expected output**: although it may not match perfectly, your expected output should be close to ours and your cost value should decrease.

```python
Cost after epoch 0 = 1.917929
Cost after epoch 5 = 1.506757
Train Accuracy     = 0.940741
Test Accuracy      = 0.783333
```

<div class="alert alert-success">
    <p>Awesome! You just built (possibly) your first TensorFlow model, which recognizes 0-5 in ASL with 80% accuracy (on the testing set)! If you want, go ahead and play around with the dataset some more – you can improve the accuruacy by spending more time tuning hypyterparameters, or even using regularization (as there's clearly high variance in this model).</div>

Once again, here's a thumbs up for your work! 

In [ ]:
fname = "images/surprise.jpg"
image = np.array(ndimage.imread(fname, flatten=False))
my_image = scipy.misc.imresize(image, size=(64,64))
plt.imshow(my_image)